In [92]:
# train_label데이터는 단일속성이기 때문에 group_by연산이나 EDA시 주의할것
# train_trade 형변환 완전히 안해줌!!!
%matplotlib inline
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
import mglearn
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 50) #칼럼 표시개수 설정
#pd.set_option('display.max_rows', 100) #로우 표시개수 설정
#plt.rcParams['agg.path.chunksize'] = 10000 #그래프 용량
#data[data['acc_id'].isin(ne[ne[0] == 2]['acc_id'])] 조건검색

# 생존90만 / 이탈60만

# rich_monster == boss_monster?? 칼럼명 바뀐건가?
# correlation 중복된거 빼고

train_activity = pd.read_csv("C:/Users/SS/Desktop/data/train_activity.csv")
train_combat = pd.read_csv("C:/Users/SS/Desktop/data/train_combat.csv")
train_label = pd.read_csv("C:/Users/SS/Desktop/data/train_label.csv") #일별 평균 결제금액
train_payment = pd.read_csv("C:/Users/SS/Desktop/data/train_payment.csv")
train_pledge = pd.read_csv("C:/Users/SS/Desktop/data/train_pledge.csv")
train_trade = pd.read_csv("C:/Users/SS/Desktop/data/train_trade.csv")

test1_activity = pd.read_csv("C:/Users/SS/Desktop/data/test1_activity.csv")
test1_combat = pd.read_csv("C:/Users/SS/Desktop/data/test1_combat.csv")
test1_payment = pd.read_csv("C:/Users/SS/Desktop/data/test1_payment.csv")
test1_pledge = pd.read_csv("C:/Users/SS/Desktop/data/test1_pledge.csv")
test1_trade = pd.read_csv("C:/Users/SS/Desktop/data/test1_trade.csv")

test2_activity = pd.read_csv("C:/Users/SS/Desktop/data/test2_activity.csv")
test2_combat = pd.read_csv("C:/Users/SS/Desktop/data/test2_combat.csv")
test2_payment = pd.read_csv("C:/Users/SS/Desktop/data/test2_payment.csv")
test2_pledge = pd.read_csv("C:/Users/SS/Desktop/data/test2_pledge.csv")
test2_trade = pd.read_csv("C:/Users/SS/Desktop/data/test2_trade.csv")

# trade 병합안함
train_trade['item_price'].fillna(0 , inplace =True) #교환창거래 == 0.0000

train_pledge.rename({'random_attacker_cnt': 'random_attacker_cnt_p', 'random_defender_cnt': 'random_defender_cnt_p', 
                     'same_pledge_cnt' : 'same_pledge_cnt_p','temp_cnt' : 'temp_cnt_p',  
                     'etc_cnt' : 'etc_cnt_p'   }, axis=1, inplace=True) # 겹치는 칼럼명 이름변경

train_payment.columns = ['day', 'acc_id', 'day_spent'] # amount_spent의 칼럼 명이 겹치므로 변경, 나중에 test1,2 데이터도 똑같이 변경!

m = pd.merge(train_activity, train_label, on = 'acc_id')
m = pd.merge(m, train_combat, on = ['day','acc_id','char_id','server'], how = 'left')
m = pd.merge(m, train_pledge, on = ['day','acc_id','char_id','server'], how = 'left') 
m = pd.merge(m, train_payment, on = ['day','acc_id'], how = 'left')
data = m

#형변환
#data['day'] = data['day'].astype(str)
data['acc_id'] = data['acc_id'].astype(str)
data['char_id'] = data['char_id'].astype(str)
data['rich_monster'] = data['rich_monster'].astype(str)
data['class'] = data['class'].astype(str)
data['level'] = data['level'].astype(str)
data['pledge_id'] = data['pledge_id'].astype(str)

train_trade['source_acc_id'] = train_trade['source_acc_id'].astype(str)
train_trade['target_acc_id'] = train_trade['target_acc_id'].astype(str)
train_trade['source_char_id'] = train_trade['source_char_id'].astype(str)
train_trade['target_char_id'] = train_trade['target_char_id'].astype(str)

# 생존 / 비생존 집단 구분
data['suv'] = data['survival_time'].apply(lambda x : 'suv' if x == 64 else 'no_suv')

#co = data.groupby('acc_id').sum().reset_index().corr()['amount_spent'] #유저별 변수 합하고나서 survival_time/ amount_spent과의 상관계수
#co[co>0.1]

#sns.boxplot(x="suv", y="playtime", data=data) #유저별 캐릭터별 play시간 합치지 않고 본것
data['day_spent'].fillna(0 , inplace =True) 

data['pledge_id'] = data['pledge_id'].apply(lambda x : np.nan if x =='nan' else x) #nan에서 진짜 NAN값으로 바꿔주기
data['class'] = data['class'].apply(lambda x : np.nan if x =='nan' else x)
data['level'] = data['level'].apply(lambda x : np.nan if x =='nan' else x)

data['play_char_cnt'].fillna(0 , inplace =True)
data['combat_char_cnt'].fillna(0 , inplace =True) 
data['pledge_combat_cnt'].fillna(0 , inplace =True) 
data['random_attacker_cnt_p'].fillna(0 , inplace =True) 
data['random_defender_cnt_p'].fillna(0 , inplace =True) 
data['same_pledge_cnt_p'].fillna(0 , inplace =True)
data['temp_cnt_p'].fillna(0 , inplace =True)
data['etc_cnt_p'].fillna(0 , inplace =True)
data['combat_play_time'].fillna(0 , inplace =True)
data['non_combat_play_time'].fillna(0 , inplace =True)
data['pledge_cnt'].fillna(0 , inplace =True)
data['random_attacker_cnt'].fillna(0 , inplace =True)
data['random_defender_cnt'].fillna(0 , inplace =True)
data['non_combat_play_time'].fillna(0 , inplace =True)
data['temp_cnt'].fillna(0 , inplace =True)
data['same_pledge_cnt'].fillna(0 , inplace =True)
data['etc_cnt'].fillna(0 , inplace =True)
data['num_opponent'].fillna(0 , inplace =True)

#pledge_id가 NAN인 데이터 == 혈맹활동이 없었던 날
#data[data['class'].isnull()] -> class, level이 NAN인날 -> 전투활동이 없었던 날

#------------------------------------------------------------------------------------
# 나중에 할것
# play횟수에 따라서 차이나는 생존률 25?26?일부터 suv의 숫자가 no_suv의 숫자보다 많아지기 시작한다.
#a= data[['day','acc_id']].drop_duplicates().groupby('acc_id').count().reset_index()
#a[a['day'] <2]
#data[data['acc_id'].isin(a[a['day'] ==28]['acc_id'])]['suv'].value_counts()


%matplotlib inline
from sklearn.preprocessing import StandardScaler

li = data.columns.tolist()
li.remove('day')
li.remove('acc_id')
li.remove('char_id')
li.remove('server')
li.remove('rich_monster')
li.remove('class' )
li.remove('level')
li.remove('pledge_id')
li.remove('suv')
clus = data

x = clus.groupby('acc_id').mean().reset_index()[li].dropna().values #drop_duplicates빼 sum으로 해야하나?
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x) # 데이터 표준화 standardization
'''
sum = []
for i in range(2,20) :
    
        Kmeans = KMeans(n_clusters = i)
        Kmeans.fit(x)

        sum.append(Kmeans.inertia_)
        
plt.plot(sum)
'''
Kmeans = KMeans(n_clusters = 10 , n_init = 30)
Kmeans.fit(x_scaled)

clusdata = pd.DataFrame(Kmeans.cluster_centers_)
clusdata.rename({0:'playtime', 1:'npc_kill',
      2: 'solo_exp', 3: 'party_exp', 4:'quest_exp',5: 'death', 6:'revive',
      7: 'exp_recovery',8: 'fishing', 9:'private_shop', 10: 'game_money_change',
      11: 'enchant_count',12:'survival_time' , 13:'amount_spent', 14:'pledge_cnt', 15:'random_attacker_cnt', 16:'random_defender_cnt', 17:'temp_cnt', 18:'same_pledge_cnt', 19:'etc_cnt', 20:'num_opponent', 21:'play_char_cnt'
                 , 22: 'combat_char_cnt' , 23:'pledge_combat_cnt', 24:'random_attacker_cnt_p', 25:'random_defender_cnt_p',
                26:'same_pledge_cnt_p', 27:'temp_cnt_p', 28:'etc_cnt_p', 29:'combat_play_time' ,30:'non_combat_play_time',31:'day_spent',32:'days'}, axis =1 , inplace= True)

#print(clusdata)
#print()
print(pd.Series(Kmeans.labels_).value_counts()) #아웃라이어(비생존) 발견!!!!!!!!!'''

0    18295
7     8418
5     4315
8     4270
1     1919
4     1638
3     1075
6       57
9       12
2        1
dtype: int64


In [93]:
clustering = pd.DataFrame(Kmeans.labels_)
clustering.rename({0:'mean_clustering'}, axis =1 , inplace = True)

li.append('acc_id')
anomaly = pd.concat([clus.groupby('acc_id').mean().reset_index()[li].dropna(), clustering] , axis=1 )
anomaly[['acc_id','mean_clustering']]

ano = pd.merge(data,anomaly[['acc_id','mean_clustering']]).groupby('mean_clustering').mean().reset_index()

anomaly_acc_id = anomaly[(anomaly['mean_clustering'] == 2) | (anomaly['mean_clustering'] == 9)]['acc_id'] #이상 군집 제거
#anomaly_acc_id

#data = data[~(data['acc_id'].isin(anomaly_acc_id))]

#plt.figure(figsize = (15,15)) #각 군집별 특징 barplot
#for i in ano.columns[2:] :
             
#             ano[[i]].plot(kind = 'bar',figsize=(10, 5))

trade_id = pd.concat([train_trade['source_acc_id'] ,train_trade['target_acc_id']]).drop_duplicates()
no_trade_id = data[~(data['acc_id'].isin(trade_id))] #약 6000여명이 거래데이터가 없음 ->거래가 없는 데이터들

#no_plege_trade = no_trade_id[no_trade_id['pledge_id'].isnull()] # ->거래x,혈맹x 

leastpledge = no_trade_id[~(no_trade_id['pledge_id'].isnull())]['acc_id'].drop_duplicates() # 혈맹에 단 한군데도 가입되지 않은 유저 걸러내기, 하나라도 혈맹에 든애들
no_trade_pledge = no_trade_id[~(no_trade_id['acc_id'].isin(leastpledge))]

# 긴 생존기간 동안 playtime이 3사분위수 이상,사회적 거래x, 혈맹x, 소비x 데이터 삭제
outlier = no_trade_pledge.groupby('acc_id').mean().reset_index()
outlier = outlier[(outlier['playtime'] > 1.76) & (outlier['npc_kill'] > 0.71) & (outlier['survival_time'] > 60)] #survival_time뺴야겠다.

data = data[~(data['acc_id'].isin(outlier['acc_id']))]


t = pd.DataFrame(data['acc_id'].isin(trade_id))
t = t.rename({'acc_id' : 'trade'} , axis = 1 )
t['trade'] = t['trade'].apply(lambda x : 'Y' if x == True else 'N')
data = pd.concat([data , t] , axis =1 )

# 특정 날 , 특정 서버 , 특정 유저의 상대방에게 주는 아이템의 양과 그에 따른 수익의 합
source = train_trade.groupby(['day', 'server','source_acc_id','source_char_id']).sum().reset_index() 
source = source.rename({'source_acc_id' : 'acc_id', 'source_char_id' : 'char_id', 'item_amount' : 'give_item_amount',
              'item_price' : 'revenue_item_price'}, axis =1 )
source = source.drop(['type'], axis =1 )

data = pd.merge(data, source, on = ['day','server','acc_id','char_id'], how = 'left')

# 특정 날 , 특정 서버 , 특정 유저의 상대방에게 받는 아이템의 양과 그에 따른 수익의 합
target = train_trade.groupby(['day', 'server','target_acc_id','target_char_id']).sum().reset_index() 
target = target.rename({'target_acc_id' : 'acc_id', 'target_char_id' : 'char_id', 'item_amount' : 'receive_item_amount',
              'item_price' : 'spend_item_price'}, axis =1 )
target = target.drop(['type'], axis =1 )

data = pd.merge(data, target, on = ['day','server','acc_id','char_id'], how = 'left')

#NA값 채워주기

data['give_item_amount'].fillna(0, inplace = True) 
data['receive_item_amount'].fillna(0, inplace = True)

data['revenue_item_price'].fillna(0, inplace = True)
data['spend_item_price'].fillna(0, inplace = True)

#총 거래량
data['total_item_amount'] = data['give_item_amount'] + data['receive_item_amount']


tem = train_trade.groupby(['day','source_acc_id']).nunique()
tem = tem.drop(['source_acc_id','day'], axis = 1 ) #target_acc_id의 수
tem = tem.reset_index()

tem1 = train_trade.groupby(['day','target_acc_id']).nunique()
tem1 = tem1.drop(['target_acc_id','day'], axis = 1 ) #source_acc_id의 수
tem1 = tem1.reset_index()

tem = tem.rename({'source_acc_id' : 'acc_id'}, axis =1 )[['day','acc_id','target_acc_id']] #target_acc_id의 수
tem1 = tem1.rename({'target_acc_id' : 'acc_id'}, axis =1 )[['day','acc_id','source_acc_id']]

numkindpartner = pd.merge(tem , tem1 , on = ['day', 'acc_id'], how = 'outer')

numkindpartner['target_acc_id'].fillna(0, inplace= True)
numkindpartner['source_acc_id'].fillna(0, inplace= True)
numkindpartner['total_kind_trade_partner'] = numkindpartner['target_acc_id'] + numkindpartner['source_acc_id']

numkindpartner = numkindpartner.groupby('acc_id').var().reset_index()
numkindpartner['total_kind_trade_partner'].fillna(0, inplace = True)

numkindpartner = numkindpartner.rename({'total_kind_trade_partner' : 'var_kindpartner_day'}, axis =1)
numkindpartner = numkindpartner[['acc_id','var_kindpartner_day']]

data = pd.merge(data , numkindpartner, on= 'acc_id', how = 'left')
data['var_kindpartner_day'].fillna(0 ,inplace = True)


give = train_trade.groupby(['day','source_acc_id']).count().reset_index()[['day','source_acc_id','target_acc_id']]
take = train_trade.groupby(['day','target_acc_id']).count().reset_index()[['day','target_acc_id','source_acc_id']]

give = give.rename({'source_acc_id' : 'acc_id'}, axis =1 )
take = take.rename({'target_acc_id' : 'acc_id'}, axis =1 )

trade_mean = pd.merge(give , take , on = ['day', 'acc_id'], how = 'outer')
trade_mean['target_acc_id'].fillna(0, inplace = True)
trade_mean['source_acc_id'].fillna(0, inplace = True)

trade_mean['trade_mean'] = trade_mean['target_acc_id'] + trade_mean['source_acc_id']

trade_mean = trade_mean.groupby('acc_id').mean().reset_index()[['acc_id','trade_mean']]
data = pd.merge(data, trade_mean, on = 'acc_id', how = 'left')
data['trade_mean'].fillna(0, inplace = True)


plepar = data.groupby(['day','acc_id']).count().reset_index()[['day','acc_id','pledge_id']]

plepar = plepar.groupby('acc_id').sum().reset_index()
plepar.rename({'pledge_id' : 'total_parti_pledge'}, axis =1 , inplace = True)
plepar = plepar.drop(['day'], axis = 1)

data = pd.merge(data , plepar , on= 'acc_id' , how = 'left')

'''
data1 = data[['acc_id','char_id','server','suv','pledge_id']] # 캐릭터 고려함
data1 = data1.drop_duplicates()
data1 = data1.dropna(axis = 0)
# in - out 으로 생존/이탈 표시  0 = 생존 , 1 = 이탈

data1['IN-OUT'] = data1['suv'].apply(lambda x : 1 if x == 'no_suv' else 0)
aa = data1.groupby('pledge_id').sum().reset_index() #IN-OUT -> 이탈한 유저의 수
bb = data1.groupby('pledge_id').size().to_frame().reset_index().rename({0:'num'} , axis =1 )

cc = pd.merge(aa,bb)
cc['pledge_churn_ratio'] = cc['IN-OUT'] / cc['num']

cc = cc[['pledge_id','pledge_churn_ratio']] # nan / 0.422901 # nan 0으로 처리?????????????????????????????????????
cc.loc[21860,'pledge_churn_ratio'] = np.nan                  # 혈맹원수가 극히 적은것들 쳐내야하나??????????????????

data = pd.merge(data, cc)'''


mean_playtime = data.groupby(['acc_id']).mean().reset_index()[['acc_id','playtime']]
mean_playtime.rename({'playtime' : 'mean_playtime'} , axis =1, inplace = True )
data = pd.merge(data, mean_playtime, on ='acc_id', how = 'left')


pledge_num = data[['acc_id','pledge_id']].drop_duplicates().dropna().groupby('acc_id').count().reset_index()
pledge_num.rename({'pledge_id' : 'pledge_num'} , axis =1, inplace = True )
data =pd.merge(data, pledge_num, on= 'acc_id' , how = 'left')


pledge_size = data[['acc_id','pledge_id']].drop_duplicates().dropna().groupby('pledge_id').count().reset_index()
pledge_size.rename({'acc_id' : 'member_num'}, axis= 1 , inplace = True)

mem = data[['acc_id','pledge_id']].drop_duplicates().dropna()
mem = pd.merge(mem, pledge_size, on ='pledge_id', how = 'left')
mem = mem.groupby('acc_id').sum().reset_index()

data = pd.merge(data, mem , on='acc_id', how = 'left')

data['pledge_num'].fillna(0, inplace= True)
data['member_num'].fillna(0, inplace= True)


con = data[['day','acc_id']].drop_duplicates() # 얼마나 몰아서 했는가

con['Data_lagged'] = con.groupby(['acc_id'])['day'].shift(1)
con['diff'] = con['day'] - con['Data_lagged']
con.fillna(0 ,inplace= True)

con = con[con['diff']<=1]

con = con.groupby('acc_id').sum().reset_index()
con['ratioConsecutive'] = con['diff'] / 27 # 27?????????????????????????????????????????????????????????????????????
con.drop(['day','Data_lagged','diff'], axis =1 , inplace = True)

data = pd.merge(data, con, on = ['acc_id'])


tot = data.groupby('acc_id').sum().reset_index()
tot = tot[['acc_id','playtime']]
tot.rename({'playtime' : 'total_playtime'}, axis = 1, inplace= True)

data = pd.merge(data, tot, on = ['acc_id'])

#sns.boxplot(  y= 'total_playtime' , x= "suv", data= data)


timedelta = data.groupby(['day','acc_id']).sum().reset_index() # 유저별 캐릭터별 total playtime 
timedelta = timedelta[['day','acc_id','playtime']]
timedelta['day_lagged'] = timedelta.groupby(['acc_id'])['day'].shift(1)
timedelta['total_playtime_lagged'] = timedelta.groupby(['acc_id'])['playtime'].shift(1)

timedelta['day_diff'] = timedelta['day'] - timedelta['day_lagged']
timedelta['playtime_diff'] = timedelta['playtime'] - timedelta['total_playtime_lagged']
#.timedelta[timedelta['acc_id'] == '75001']

timedelta = timedelta.groupby('acc_id').sum().reset_index()
timedelta['total_playtime_changeAve'] = timedelta['playtime_diff'] / timedelta['day_diff']
timedelta = timedelta.fillna(0)[['acc_id','total_playtime_changeAve']]

data = pd.merge(data, timedelta, on = ['acc_id'])
#sns.boxplot(  y= 'total_playtime_changeAve' , x= "suv", data= data)

# 변화1 + 변화2 + 변화3 / 3

'''
combatvar = data[['acc_id','pledge_cnt','temp_cnt','same_pledge_cnt','etc_cnt']].groupby('acc_id').var().reset_index()
combatvar['combatvar'] = combatvar['pledge_cnt'] + combatvar['temp_cnt'] + combatvar['same_pledge_cnt'] + combatvar['etc_cnt']
combatvar = combatvar[['acc_id','combatvar']]

data = pd.merge(data, combatvar)'''


source_type = train_trade.groupby('source_acc_id').sum().reset_index()[['source_acc_id','type']] #교환창 총 거래횟수
target_type = train_trade.groupby('target_acc_id').sum().reset_index()[['target_acc_id','type']]

target_type.rename({'target_acc_id' : 'acc_id', 'type' : 'type1'} , axis= 1 , inplace = True)
source_type.rename({'source_acc_id' : 'acc_id'} , axis= 1 , inplace = True)

exchange = pd.merge(target_type, source_type, on = 'acc_id', how = 'outer')
exchange['type'].fillna(0, inplace = True)
exchange['type1'].fillna(0, inplace = True)

exchange['num_exchange'] = exchange['type1'] + exchange['type']

exchange = exchange[['acc_id','num_exchange']]
data = pd.merge(data, exchange, on = 'acc_id', how = 'left')
data['num_exchange'].fillna(0, inplace = True)

################
total_source_type = train_trade.groupby('source_acc_id').count().reset_index()[['source_acc_id','type']] #총거래수
total_target_type = train_trade.groupby('target_acc_id').count().reset_index()[['target_acc_id','type']]

total_target_type.rename({'target_acc_id' : 'acc_id', 'type' : 'type1'} , axis= 1 , inplace = True)
total_source_type.rename({'source_acc_id' : 'acc_id'} , axis= 1 , inplace = True)

total_trade = pd.merge(total_source_type, total_target_type, on ='acc_id', how = 'outer')
total_trade['type'].fillna(0, inplace = True)
total_trade['type1'].fillna(0, inplace = True)

total_trade['total_trade'] = total_trade['type'] + total_trade['type1']
total_trade = total_trade[['acc_id','total_trade']]

data = pd.merge(data , total_trade, on='acc_id' , how = 'left')
data['total_trade'].fillna(0, inplace = True)

data['num_private_shop'] = data['total_trade'] - data['num_exchange'] #개인샵 거래횟수의 총합
data['ratio_num_exchange'] = data['num_exchange'] / data['total_trade']   #총 거래중 교환창 거래의 비율 
data['ratio_num_private_shop'] = data['num_private_shop'] / data['total_trade'] #총 거래중 개인샵 거래의 비율

data['ratio_num_exchange'].fillna(0, inplace = True)
data['ratio_num_private_shop'].fillna(0, inplace = True)

days = data[['day','acc_id']].drop_duplicates().groupby('acc_id').count().reset_index()
days.rename({'day' : 'days'} , axis =1 , inplace = True)
data = pd.merge(data, days)


# 각 유저마다 특정 혈맹에 연속적으로 참여하는 비율을 구하고 그 비율을 특정혈맹으로 묶어 평균 (특정 혈맹에 연속적으로 참여하는 평균 비율)
# 내가 속한 혈맹의 참여율
pledgeConsecutive = data[['day','acc_id','pledge_id']].drop_duplicates().dropna() 
pledgeConsecutive['day_lagged'] = pledgeConsecutive.groupby(['acc_id','pledge_id'])['day'].shift(1)
#pledgeConsecutive[(pledgeConsecutive['acc_id'] == '75001') & (pledgeConsecutive['pledge_id'] == '12443.0')]

pledgeConsecutive['diff'] = pledgeConsecutive['day'] - pledgeConsecutive['day_lagged']
pledgeConsecutive['diff'].fillna(0, inplace = True)
pledgeConsecutive = pledgeConsecutive[pledgeConsecutive['diff']<=1]
pledgeConsecutive = pledgeConsecutive.groupby(['acc_id','pledge_id']).sum().reset_index()

pledgeConsecutive['pledge_ratioConsecutive'] = pledgeConsecutive['diff'] / 27
pledgeConsecutive = pledgeConsecutive[['acc_id','pledge_id','pledge_ratioConsecutive']]

pledgeConsecutive = pledgeConsecutive.groupby('pledge_id').mean().reset_index()

data = pd.merge(data , pledgeConsecutive, on ='pledge_id', how = 'left')

mean_pledge_ratioConsecutive = data[['acc_id','pledge_id','pledge_ratioConsecutive']].drop_duplicates().dropna()
mean_pledge_ratioConsecutive = mean_pledge_ratioConsecutive.groupby('acc_id').mean().reset_index()

mean_pledge_ratioConsecutive.rename({'pledge_ratioConsecutive' : 'mean_pledge_ratioConsecutive'}, axis =1 , inplace = True)
data = pd.merge(data , mean_pledge_ratioConsecutive, on ='acc_id', how ='left')
data['mean_pledge_ratioConsecutive'].fillna(0, inplace = True)


# 유저당 캐릭터 개수
char = data[['acc_id','char_id']].drop_duplicates().groupby('acc_id').count().reset_index()
char.rename({'char_id': 'num_char'}, axis =1 , inplace = True)
data = pd.merge(data , char , on ='acc_id' , how ='left')

#플레이 횟수
playnum = data.groupby('acc_id').count().reset_index()[['acc_id','day']]
playnum.rename({'day' : 'playnum'} , axis =1 , inplace = True)
data = pd.merge(data , playnum , on  = 'acc_id', how = 'left')

# 몬스터 타격 횟수
data['rich_monster'] = data['rich_monster'].astype(int)

rich_monster = data.groupby('acc_id').sum().reset_index()[['acc_id','rich_monster']]
rich_monster.rename({'rich_monster' : 'num_rich_monster'}, axis=1 ,inplace = True)
data = pd.merge(data, rich_monster , on ='acc_id', how = 'left')


# level 변화의 총합

data['level'] = data['level'].astype(float)
a = data[['acc_id','char_id','level']]
level_max = a.groupby(['acc_id','char_id']).max().reset_index()
level_max.rename({'level' : 'max_level'} , axis =1 , inplace = True)
level_min = a.groupby(['acc_id','char_id']).min().reset_index()
level_min.rename({'level' : 'min_level'} , axis =1 , inplace = True)

level = pd.merge(level_max, level_min, on =['acc_id','char_id']) 
level['level_change'] = level['max_level'] - level['min_level']
level = level[['acc_id','char_id','level_change']]
level = level.groupby('acc_id').sum().reset_index()

data = pd.merge(data, level, on ='acc_id', how = 'left')

# TestData1

In [94]:
test1_trade['item_price'].fillna(0 , inplace =True) #교환창거래 == 0.0000

test1_pledge.rename({'random_attacker_cnt': 'random_attacker_cnt_p', 'random_defender_cnt': 'random_defender_cnt_p', 
                     'same_pledge_cnt' : 'same_pledge_cnt_p','temp_cnt' : 'temp_cnt_p',  
                     'etc_cnt' : 'etc_cnt_p'   }, axis=1, inplace=True) # 겹치는 칼럼명 이름변경

test1_payment.columns = ['day', 'acc_id', 'day_spent'] # amount_spent의 칼럼 명이 겹치므로 변경, 나중에 test1,2 데이터도 똑같이 변경!

#m = pd.merge(train_activity, train_label, on = 'acc_id')
m = pd.merge(test1_activity , test1_combat, on = ['day','acc_id','char_id','server'], how = 'left')
m = pd.merge(m, test1_pledge, on = ['day','acc_id','char_id','server'], how = 'left') 
m = pd.merge(m, test1_payment, on = ['day','acc_id'], how = 'left')
testdata1 = m

#형변환
#data['day'] = data['day'].astype(str)
testdata1['acc_id'] = testdata1['acc_id'].astype(str)
testdata1['char_id'] = testdata1['char_id'].astype(str)
testdata1['rich_monster'] = testdata1['rich_monster'].astype(str)
testdata1['class'] = testdata1['class'].astype(str)
testdata1['level'] = testdata1['level'].astype(str)
testdata1['pledge_id'] = testdata1['pledge_id'].astype(str)

test1_trade['source_acc_id'] = test1_trade['source_acc_id'].astype(str)
test1_trade['target_acc_id'] = test1_trade['target_acc_id'].astype(str)
test1_trade['source_char_id'] = test1_trade['source_char_id'].astype(str)
test1_trade['target_char_id'] = test1_trade['target_char_id'].astype(str)

# 생존 / 비생존 집단 구분
#data['suv'] = data['survival_time'].apply(lambda x : 'suv' if x == 64 else 'no_suv')

#co = data.groupby('acc_id').sum().reset_index().corr()['amount_spent'] #유저별 변수 합하고나서 survival_time/ amount_spent과의 상관계수
#co[co>0.1]

#sns.boxplot(x="suv", y="playtime", data=data) #유저별 캐릭터별 play시간 합치지 않고 본것
testdata1['day_spent'].fillna(0 , inplace =True) 

testdata1['pledge_id'] = testdata1['pledge_id'].apply(lambda x : np.nan if x =='nan' else x) #nan에서 진짜 NAN값으로 바꿔주기
testdata1['class'] = testdata1['class'].apply(lambda x : np.nan if x =='nan' else x)
testdata1['level'] = testdata1['level'].apply(lambda x : np.nan if x =='nan' else x)

testdata1['play_char_cnt'].fillna(0 , inplace =True)
testdata1['combat_char_cnt'].fillna(0 , inplace =True) 
testdata1['pledge_combat_cnt'].fillna(0 , inplace =True) 
testdata1['random_attacker_cnt_p'].fillna(0 , inplace =True) 
testdata1['random_defender_cnt_p'].fillna(0 , inplace =True) 
testdata1['same_pledge_cnt_p'].fillna(0 , inplace =True)
testdata1['temp_cnt_p'].fillna(0 , inplace =True)
testdata1['etc_cnt_p'].fillna(0 , inplace =True)
testdata1['combat_play_time'].fillna(0 , inplace =True)
testdata1['non_combat_play_time'].fillna(0 , inplace =True)
testdata1['pledge_cnt'].fillna(0 , inplace =True)
testdata1['random_attacker_cnt'].fillna(0 , inplace =True)
testdata1['random_defender_cnt'].fillna(0 , inplace =True)
testdata1['non_combat_play_time'].fillna(0 , inplace =True)
testdata1['temp_cnt'].fillna(0 , inplace =True)
testdata1['same_pledge_cnt'].fillna(0 , inplace =True)
testdata1['etc_cnt'].fillna(0 , inplace =True)
testdata1['num_opponent'].fillna(0 , inplace =True)



#days
days = testdata1[['day','acc_id']].drop_duplicates().groupby('acc_id').count().reset_index()
days.rename({'day' : 'days'} , axis =1 , inplace = True)
testdata1 = pd.merge(testdata1, days)


'''
trade_id = pd.concat([test1_trade['source_acc_id'] ,test1_trade['target_acc_id']]).drop_duplicates()
no_trade_id = testdata1[~(testdata1['acc_id'].isin(trade_id))] #약 6000여명이 거래데이터가 없음 ->거래가 없는 데이터들

#no_plege_trade = no_trade_id[no_trade_id['pledge_id'].isnull()] # ->거래x,혈맹x 

leastpledge = no_trade_id[~(no_trade_id['pledge_id'].isnull())]['acc_id'].drop_duplicates() # 혈맹에 단 한군데도 가입되지 않은 유저 걸러내기, 하나라도 혈맹에 든애들
no_trade_pledge = no_trade_id[~(no_trade_id['acc_id'].isin(leastpledge))]

# 긴 생존기간 동안 playtime이 3사분위수 이상,사회적 거래x, 혈맹x, 소비x 데이터 삭제
outlier = no_trade_pledge.groupby('acc_id').mean().reset_index()
outlier = outlier[(outlier['playtime'] > 2.35) & (outlier['npc_kill'] > 0.39) ]

testdata1 = testdata1[~(testdata1['acc_id'].isin(outlier['acc_id']))]'''


# 거래유무 변수
t = pd.DataFrame(testdata1['acc_id'].isin(trade_id))
t = t.rename({'acc_id' : 'trade'} , axis = 1 )
t['trade'] = t['trade'].apply(lambda x : 'Y' if x == True else 'N')
testdata1 = pd.concat([testdata1 , t] , axis =1 )


# 특정 날 , 특정 서버 , 특정 유저의 상대방에게 주는 아이템의 양과 그에 따른 수익의 합
source = test1_trade.groupby(['day', 'server','source_acc_id','source_char_id']).sum().reset_index() 
source = source.rename({'source_acc_id' : 'acc_id', 'source_char_id' : 'char_id', 'item_amount' : 'give_item_amount',
              'item_price' : 'revenue_item_price'}, axis =1 )
source = source.drop(['type'], axis =1 )

testdata1 = pd.merge(testdata1, source, on = ['day','server','acc_id','char_id'], how = 'left')

# 특정 날 , 특정 서버 , 특정 유저의 상대방에게 받는 아이템의 양과 그에 따른 수익의 합
target = test1_trade.groupby(['day', 'server','target_acc_id','target_char_id']).sum().reset_index() 
target = target.rename({'target_acc_id' : 'acc_id', 'target_char_id' : 'char_id', 'item_amount' : 'receive_item_amount',
              'item_price' : 'spend_item_price'}, axis =1 )
target = target.drop(['type'], axis =1 )

testdata1 = pd.merge(testdata1, target, on = ['day','server','acc_id','char_id'], how = 'left')

#NA값 채워주기

testdata1['give_item_amount'].fillna(0, inplace = True) 
testdata1['receive_item_amount'].fillna(0, inplace = True)

testdata1['revenue_item_price'].fillna(0, inplace = True)
testdata1['spend_item_price'].fillna(0, inplace = True)

#총 거래량
testdata1['total_item_amount'] = testdata1['give_item_amount'] + testdata1['receive_item_amount']




tem = test1_trade.groupby(['day','source_acc_id']).nunique()
tem = tem.drop(['source_acc_id','day'], axis = 1 ) #target_acc_id의 수
tem = tem.reset_index()

tem1 = test1_trade.groupby(['day','target_acc_id']).nunique()
tem1 = tem1.drop(['target_acc_id','day'], axis = 1 ) #source_acc_id의 수
tem1 = tem1.reset_index()

tem = tem.rename({'source_acc_id' : 'acc_id'}, axis =1 )[['day','acc_id','target_acc_id']] #target_acc_id의 수
tem1 = tem1.rename({'target_acc_id' : 'acc_id'}, axis =1 )[['day','acc_id','source_acc_id']]

numkindpartner = pd.merge(tem , tem1 , on = ['day', 'acc_id'], how = 'outer')

numkindpartner['target_acc_id'].fillna(0, inplace= True)
numkindpartner['source_acc_id'].fillna(0, inplace= True)
numkindpartner['total_kind_trade_partner'] = numkindpartner['target_acc_id'] + numkindpartner['source_acc_id']

numkindpartner = numkindpartner.groupby('acc_id').var().reset_index()
numkindpartner['total_kind_trade_partner'].fillna(0, inplace = True)

numkindpartner = numkindpartner.rename({'total_kind_trade_partner' : 'var_kindpartner_day'}, axis =1)
numkindpartner = numkindpartner[['acc_id','var_kindpartner_day']]

testdata1 = pd.merge(testdata1 , numkindpartner, on= 'acc_id', how = 'left')
testdata1['var_kindpartner_day'].fillna(0 ,inplace = True)

#######################
give = test1_trade.groupby(['day','source_acc_id']).count().reset_index()[['day','source_acc_id','target_acc_id']]
take = test1_trade.groupby(['day','target_acc_id']).count().reset_index()[['day','target_acc_id','source_acc_id']]

give = give.rename({'source_acc_id' : 'acc_id'}, axis =1 )
take = take.rename({'target_acc_id' : 'acc_id'}, axis =1 )

trade_mean = pd.merge(give , take , on = ['day', 'acc_id'], how = 'outer')
trade_mean['target_acc_id'].fillna(0, inplace = True)
trade_mean['source_acc_id'].fillna(0, inplace = True)

trade_mean['trade_mean'] = trade_mean['target_acc_id'] + trade_mean['source_acc_id']

trade_mean = trade_mean.groupby('acc_id').mean().reset_index()[['acc_id','trade_mean']]
testdata1 = pd.merge(testdata1, trade_mean, on = 'acc_id', how = 'left')
testdata1['trade_mean'].fillna(0, inplace = True)


############################
plepar = testdata1.groupby(['day','acc_id']).count().reset_index()[['day','acc_id','pledge_id']]

plepar = plepar.groupby('acc_id').sum().reset_index()
plepar.rename({'pledge_id' : 'total_parti_pledge'}, axis =1 , inplace = True)
plepar = plepar.drop(['day'], axis = 1)

testdata1 = pd.merge(testdata1 , plepar , on= 'acc_id' , how = 'left')
#######################

mean_playtime = testdata1.groupby(['acc_id']).mean().reset_index()[['acc_id','playtime']]
mean_playtime.rename({'playtime' : 'mean_playtime'} , axis =1, inplace = True )
testdata1 = pd.merge(testdata1, mean_playtime, on ='acc_id', how = 'left')



##########
pledge_num = testdata1[['acc_id','pledge_id']].drop_duplicates().dropna().groupby('acc_id').count().reset_index()
pledge_num.rename({'pledge_id' : 'pledge_num'} , axis =1, inplace = True )
testdata1 =pd.merge(testdata1, pledge_num, on= 'acc_id' , how = 'left')


###########

pledge_size = testdata1[['acc_id','pledge_id']].drop_duplicates().dropna().groupby('pledge_id').count().reset_index()
pledge_size.rename({'acc_id' : 'member_num'}, axis= 1 , inplace = True)

mem = testdata1[['acc_id','pledge_id']].drop_duplicates().dropna()
mem = pd.merge(mem, pledge_size, on ='pledge_id', how = 'left')
mem = mem.groupby('acc_id').sum().reset_index()

testdata1 = pd.merge(testdata1, mem , on='acc_id', how = 'left')

testdata1['pledge_num'].fillna(0, inplace= True)
testdata1['member_num'].fillna(0, inplace= True)



con = testdata1[['day','acc_id']].drop_duplicates() # 얼마나 몰아서 했는가

con['Data_lagged'] = con.groupby(['acc_id'])['day'].shift(1)
con['diff'] = con['day'] - con['Data_lagged']
con.fillna(0 ,inplace= True)

con = con[con['diff']<=1]

con = con.groupby('acc_id').sum().reset_index()
con['ratioConsecutive'] = con['diff'] / 27 # 27?????????????????????????????????????????????????????????????????????
con.drop(['day','Data_lagged','diff'], axis =1 , inplace = True)

testdata1 = pd.merge(testdata1, con, on = ['acc_id'])


tot = testdata1.groupby('acc_id').sum().reset_index()
tot = tot[['acc_id','playtime']]
tot.rename({'playtime' : 'total_playtime'}, axis = 1, inplace= True)

testdata1 = pd.merge(testdata1, tot, on = ['acc_id'])

#sns.boxplot(  y= 'total_playtime' , x= "suv", data= data)


timedelta = testdata1.groupby(['day','acc_id']).sum().reset_index() # 유저별 캐릭터별 total playtime 
timedelta = timedelta[['day','acc_id','playtime']]
timedelta['day_lagged'] = timedelta.groupby(['acc_id'])['day'].shift(1)
timedelta['total_playtime_lagged'] = timedelta.groupby(['acc_id'])['playtime'].shift(1)

timedelta['day_diff'] = timedelta['day'] - timedelta['day_lagged']
timedelta['playtime_diff'] = timedelta['playtime'] - timedelta['total_playtime_lagged']
#.timedelta[timedelta['acc_id'] == '75001']

timedelta = timedelta.groupby('acc_id').sum().reset_index()
timedelta['total_playtime_changeAve'] = timedelta['playtime_diff'] / timedelta['day_diff']
timedelta = timedelta.fillna(0)[['acc_id','total_playtime_changeAve']]

testdata1 = pd.merge(testdata1, timedelta, on = ['acc_id'])
#sns.boxplot(  y= 'total_playtime_changeAve' , x= "suv", data= data)

# 변화1 + 변화2 + 변화3 / 3

'''
combatvar = testdata1[['acc_id','pledge_cnt','temp_cnt','same_pledge_cnt','etc_cnt']].groupby('acc_id').var().reset_index()
combatvar['combatvar'] = combatvar['pledge_cnt'] + combatvar['temp_cnt'] + combatvar['same_pledge_cnt'] + combatvar['etc_cnt']
combatvar = combatvar[['acc_id','combatvar']]

testdata1 = pd.merge(testdata1, combatvar)'''


source_type = test1_trade.groupby('source_acc_id').sum().reset_index()[['source_acc_id','type']] #교환창을 통한 총 거래횟수
target_type = test1_trade.groupby('target_acc_id').sum().reset_index()[['target_acc_id','type']]

target_type.rename({'target_acc_id' : 'acc_id', 'type' : 'type1'} , axis= 1 , inplace = True)
source_type.rename({'source_acc_id' : 'acc_id'} , axis= 1 , inplace = True)

exchange = pd.merge(target_type, source_type, on = 'acc_id', how = 'outer')
exchange['type'].fillna(0, inplace = True)
exchange['type1'].fillna(0, inplace = True)

exchange['num_exchange'] = exchange['type1'] + exchange['type']

exchange = exchange[['acc_id','num_exchange']]
testdata1 = pd.merge(testdata1, exchange, on = 'acc_id', how = 'left')
testdata1['num_exchange'].fillna(0, inplace = True)

total_source_type = test1_trade.groupby('source_acc_id').count().reset_index()[['source_acc_id','type']]
total_target_type = test1_trade.groupby('target_acc_id').count().reset_index()[['target_acc_id','type']]

total_target_type.rename({'target_acc_id' : 'acc_id', 'type' : 'type1'} , axis= 1 , inplace = True)
total_source_type.rename({'source_acc_id' : 'acc_id'} , axis= 1 , inplace = True)

total_trade = pd.merge(total_source_type, total_target_type, on ='acc_id', how = 'outer')
total_trade['type'].fillna(0, inplace = True)
total_trade['type1'].fillna(0, inplace = True)

total_trade['total_trade'] = total_trade['type'] + total_trade['type1']
total_trade = total_trade[['acc_id','total_trade']]

testdata1 = pd.merge(testdata1 , total_trade, on='acc_id' , how = 'left')
testdata1['total_trade'].fillna(0, inplace = True)

testdata1['num_private_shop'] = testdata1['total_trade'] - testdata1['num_exchange']
testdata1['ratio_num_exchange'] = testdata1['num_exchange'] / testdata1['total_trade']
testdata1['ratio_num_private_shop'] = testdata1['num_private_shop'] / testdata1['total_trade']

testdata1['ratio_num_exchange'].fillna(0, inplace = True)
testdata1['ratio_num_private_shop'].fillna(0, inplace = True)


# 각 유저마다 특정 혈맹에 연속적으로 참여하는 비율을 구하고 그 비율을 특정혈맹으로 묶어 평균 (특정 혈맹에 연속적으로 참여하는 평균 비율)
# 내가 속한 혈맹의 참여율
pledgeConsecutive = testdata1[['day','acc_id','pledge_id']].drop_duplicates().dropna() 
pledgeConsecutive['day_lagged'] = pledgeConsecutive.groupby(['acc_id','pledge_id'])['day'].shift(1)
#pledgeConsecutive[(pledgeConsecutive['acc_id'] == '75001') & (pledgeConsecutive['pledge_id'] == '12443.0')]

pledgeConsecutive['diff'] = pledgeConsecutive['day'] - pledgeConsecutive['day_lagged']
pledgeConsecutive['diff'].fillna(0, inplace = True)
pledgeConsecutive = pledgeConsecutive[pledgeConsecutive['diff']<=1]
pledgeConsecutive = pledgeConsecutive.groupby(['acc_id','pledge_id']).sum().reset_index()

pledgeConsecutive['pledge_ratioConsecutive'] = pledgeConsecutive['diff'] / 27
pledgeConsecutive = pledgeConsecutive[['acc_id','pledge_id','pledge_ratioConsecutive']]

pledgeConsecutive = pledgeConsecutive.groupby('pledge_id').mean().reset_index()

testdata1 = pd.merge(testdata1 , pledgeConsecutive, on ='pledge_id', how = 'left')

mean_pledge_ratioConsecutive = testdata1[['acc_id','pledge_id','pledge_ratioConsecutive']].drop_duplicates().dropna()
mean_pledge_ratioConsecutive = mean_pledge_ratioConsecutive.groupby('acc_id').mean().reset_index()

mean_pledge_ratioConsecutive.rename({'pledge_ratioConsecutive' : 'mean_pledge_ratioConsecutive'}, axis =1 , inplace = True)
testdata1 = pd.merge(testdata1 , mean_pledge_ratioConsecutive, on ='acc_id', how ='left')
testdata1['mean_pledge_ratioConsecutive'].fillna(0, inplace = True)


# 유저당 캐릭터 개수
char = testdata1[['acc_id','char_id']].drop_duplicates().groupby('acc_id').count().reset_index()
char.rename({'char_id': 'num_char'}, axis =1 , inplace = True)
testdata1 = pd.merge(testdata1 , char , on ='acc_id' , how ='left')


#플레이 횟수
playnum = testdata1.groupby('acc_id').count().reset_index()[['acc_id','day']]
playnum.rename({'day' : 'playnum'} , axis =1 , inplace = True)
testdata1 = pd.merge(testdata1 , playnum , on  = 'acc_id', how = 'left')

# 몬스토 타격 횟수
testdata1['rich_monster'] = testdata1['rich_monster'].astype(int)

rich_monster = testdata1.groupby('acc_id').sum().reset_index()[['acc_id','rich_monster']]
rich_monster.rename({'rich_monster' : 'num_rich_monster'}, axis=1 ,inplace = True)
testdata1 = pd.merge(testdata1, rich_monster , on ='acc_id', how = 'left')


# level 변화의 총합

data['level'] = data['level'].astype(float)
a = data[['acc_id','char_id','level']]
level_max = a.groupby(['acc_id','char_id']).max().reset_index()
level_max.rename({'level' : 'max_level'} , axis =1 , inplace = True)
level_min = a.groupby(['acc_id','char_id']).min().reset_index()
level_min.rename({'level' : 'min_level'} , axis =1 , inplace = True)

level = pd.merge(level_max, level_min, on =['acc_id','char_id']) 
level['level_change'] = level['max_level'] - level['min_level']
level = level[['acc_id','char_id','level_change']]
level = level.groupby('acc_id').sum().reset_index()

data = pd.merge(data, level, on ='acc_id', how = 'left')

# TestData2

In [95]:
test2_trade['item_price'].fillna(0 , inplace =True) #교환창거래 == 0.0000

test2_pledge.rename({'random_attacker_cnt': 'random_attacker_cnt_p', 'random_defender_cnt': 'random_defender_cnt_p', 
                     'same_pledge_cnt' : 'same_pledge_cnt_p','temp_cnt' : 'temp_cnt_p',  
                     'etc_cnt' : 'etc_cnt_p'   }, axis=1, inplace=True) # 겹치는 칼럼명 이름변경

test2_payment.columns = ['day', 'acc_id', 'day_spent'] # amount_spent의 칼럼 명이 겹치므로 변경, 나중에 test1,2 데이터도 똑같이 변경!

#m = pd.merge(train_activity, train_label, on = 'acc_id')
m = pd.merge(test2_activity , test2_combat, on = ['day','acc_id','char_id','server'], how = 'left')
m = pd.merge(m, test2_pledge, on = ['day','acc_id','char_id','server'], how = 'left') 
m = pd.merge(m, test2_payment, on = ['day','acc_id'], how = 'left')
testdata2 = m

#형변환
#data['day'] = data['day'].astype(str)
testdata2['acc_id'] = testdata2['acc_id'].astype(str)
testdata2['char_id'] = testdata2['char_id'].astype(str)
testdata2['rich_monster'] = testdata2['rich_monster'].astype(str)
testdata2['class'] = testdata2['class'].astype(str)
testdata2['level'] = testdata2['level'].astype(str)
testdata2['pledge_id'] = testdata2['pledge_id'].astype(str)

test2_trade['source_acc_id'] = test2_trade['source_acc_id'].astype(str)
test2_trade['target_acc_id'] = test2_trade['target_acc_id'].astype(str)
test2_trade['source_char_id'] = test2_trade['source_char_id'].astype(str)
test2_trade['target_char_id'] = test2_trade['target_char_id'].astype(str)

# 생존 / 비생존 집단 구분
#data['suv'] = data['survival_time'].apply(lambda x : 'suv' if x == 64 else 'no_suv')

#co = data.groupby('acc_id').sum().reset_index().corr()['amount_spent'] #유저별 변수 합하고나서 survival_time/ amount_spent과의 상관계수
#co[co>0.1]

#sns.boxplot(x="suv", y="playtime", data=data) #유저별 캐릭터별 play시간 합치지 않고 본것
testdata2['day_spent'].fillna(0 , inplace =True) 

testdata2['pledge_id'] = testdata2['pledge_id'].apply(lambda x : np.nan if x =='nan' else x) #nan에서 진짜 NAN값으로 바꿔주기
testdata2['class'] = testdata2['class'].apply(lambda x : np.nan if x =='nan' else x)
testdata2['level'] = testdata2['level'].apply(lambda x : np.nan if x =='nan' else x)

testdata2['play_char_cnt'].fillna(0 , inplace =True)
testdata2['combat_char_cnt'].fillna(0 , inplace =True) 
testdata2['pledge_combat_cnt'].fillna(0 , inplace =True) 
testdata2['random_attacker_cnt_p'].fillna(0 , inplace =True) 
testdata2['random_defender_cnt_p'].fillna(0 , inplace =True) 
testdata2['same_pledge_cnt_p'].fillna(0 , inplace =True)
testdata2['temp_cnt_p'].fillna(0 , inplace =True)
testdata2['etc_cnt_p'].fillna(0 , inplace =True)
testdata2['combat_play_time'].fillna(0 , inplace =True)
testdata2['non_combat_play_time'].fillna(0 , inplace =True)
testdata2['pledge_cnt'].fillna(0 , inplace =True)
testdata2['random_attacker_cnt'].fillna(0 , inplace =True)
testdata2['random_defender_cnt'].fillna(0 , inplace =True)
testdata2['non_combat_play_time'].fillna(0 , inplace =True)
testdata2['temp_cnt'].fillna(0 , inplace =True)
testdata2['same_pledge_cnt'].fillna(0 , inplace =True)
testdata2['etc_cnt'].fillna(0 , inplace =True)
testdata2['num_opponent'].fillna(0 , inplace =True)


#days
days = testdata2[['day','acc_id']].drop_duplicates().groupby('acc_id').count().reset_index()
days.rename({'day' : 'days'} , axis =1 , inplace = True)
testdata2 = pd.merge(testdata2, days)


'''
trade_id = pd.concat([test2_trade['source_acc_id'] ,test2_trade['target_acc_id']]).drop_duplicates()
no_trade_id = testdata2[~(testdata2['acc_id'].isin(trade_id))] #약 6000여명이 거래데이터가 없음 ->거래가 없는 데이터들

#no_plege_trade = no_trade_id[no_trade_id['pledge_id'].isnull()] # ->거래x,혈맹x 

leastpledge = no_trade_id[~(no_trade_id['pledge_id'].isnull())]['acc_id'].drop_duplicates() # 혈맹에 단 한군데도 가입되지 않은 유저 걸러내기, 하나라도 혈맹에 든애들
no_trade_pledge = no_trade_id[~(no_trade_id['acc_id'].isin(leastpledge))]

# 긴 생존기간 동안 playtime이 3사분위수 이상,사회적 거래x, 혈맹x, 소비x 데이터 삭제
outlier = no_trade_pledge.groupby('acc_id').mean().reset_index()
outlier = outlier[(outlier['playtime'] > 2.35) & (outlier['npc_kill'] > 0.39) ]

testdata2 = testdata2[~(testdata2['acc_id'].isin(outlier['acc_id']))]'''


# 거래유무 변수
t = pd.DataFrame(testdata2['acc_id'].isin(trade_id))
t = t.rename({'acc_id' : 'trade'} , axis = 1 )
t['trade'] = t['trade'].apply(lambda x : 'Y' if x == True else 'N')
testdata2 = pd.concat([testdata2 , t] , axis =1 )


# 특정 날 , 특정 서버 , 특정 유저의 상대방에게 주는 아이템의 양과 그에 따른 수익의 합
source = test2_trade.groupby(['day', 'server','source_acc_id','source_char_id']).sum().reset_index() 
source = source.rename({'source_acc_id' : 'acc_id', 'source_char_id' : 'char_id', 'item_amount' : 'give_item_amount',
              'item_price' : 'revenue_item_price'}, axis =1 )
source = source.drop(['type'], axis =1 )

testdata2 = pd.merge(testdata2, source, on = ['day','server','acc_id','char_id'], how = 'left')

# 특정 날 , 특정 서버 , 특정 유저의 상대방에게 받는 아이템의 양과 그에 따른 수익의 합
target = test2_trade.groupby(['day', 'server','target_acc_id','target_char_id']).sum().reset_index() 
target = target.rename({'target_acc_id' : 'acc_id', 'target_char_id' : 'char_id', 'item_amount' : 'receive_item_amount',
              'item_price' : 'spend_item_price'}, axis =1 )
target = target.drop(['type'], axis =1 )

testdata2 = pd.merge(testdata2, target, on = ['day','server','acc_id','char_id'], how = 'left')

#NA값 채워주기

testdata2['give_item_amount'].fillna(0, inplace = True) 
testdata2['receive_item_amount'].fillna(0, inplace = True)

testdata2['revenue_item_price'].fillna(0, inplace = True)
testdata2['spend_item_price'].fillna(0, inplace = True)

#총 거래량
testdata2['total_item_amount'] = testdata2['give_item_amount'] + testdata2['receive_item_amount']




tem = test2_trade.groupby(['day','source_acc_id']).nunique()
tem = tem.drop(['source_acc_id','day'], axis = 1 ) #target_acc_id의 수
tem = tem.reset_index()

tem1 = test2_trade.groupby(['day','target_acc_id']).nunique()
tem1 = tem1.drop(['target_acc_id','day'], axis = 1 ) #source_acc_id의 수
tem1 = tem1.reset_index()

tem = tem.rename({'source_acc_id' : 'acc_id'}, axis =1 )[['day','acc_id','target_acc_id']] #target_acc_id의 수
tem1 = tem1.rename({'target_acc_id' : 'acc_id'}, axis =1 )[['day','acc_id','source_acc_id']]

numkindpartner = pd.merge(tem , tem1 , on = ['day', 'acc_id'], how = 'outer')

numkindpartner['target_acc_id'].fillna(0, inplace= True)
numkindpartner['source_acc_id'].fillna(0, inplace= True)
numkindpartner['total_kind_trade_partner'] = numkindpartner['target_acc_id'] + numkindpartner['source_acc_id']

numkindpartner = numkindpartner.groupby('acc_id').var().reset_index()
numkindpartner['total_kind_trade_partner'].fillna(0, inplace = True)

numkindpartner = numkindpartner.rename({'total_kind_trade_partner' : 'var_kindpartner_day'}, axis =1)
numkindpartner = numkindpartner[['acc_id','var_kindpartner_day']]

testdata2 = pd.merge(testdata2 , numkindpartner, on= 'acc_id', how = 'left')
testdata2['var_kindpartner_day'].fillna(0 ,inplace = True)

#######################
give = test2_trade.groupby(['day','source_acc_id']).count().reset_index()[['day','source_acc_id','target_acc_id']]
take = test2_trade.groupby(['day','target_acc_id']).count().reset_index()[['day','target_acc_id','source_acc_id']]

give = give.rename({'source_acc_id' : 'acc_id'}, axis =1 )
take = take.rename({'target_acc_id' : 'acc_id'}, axis =1 )

trade_mean = pd.merge(give , take , on = ['day', 'acc_id'], how = 'outer')
trade_mean['target_acc_id'].fillna(0, inplace = True)
trade_mean['source_acc_id'].fillna(0, inplace = True)

trade_mean['trade_mean'] = trade_mean['target_acc_id'] + trade_mean['source_acc_id']

trade_mean = trade_mean.groupby('acc_id').mean().reset_index()[['acc_id','trade_mean']]
testdata2 = pd.merge(testdata2, trade_mean, on = 'acc_id', how = 'left')
testdata2['trade_mean'].fillna(0, inplace = True)


############################
plepar = testdata2.groupby(['day','acc_id']).count().reset_index()[['day','acc_id','pledge_id']]

plepar = plepar.groupby('acc_id').sum().reset_index()
plepar.rename({'pledge_id' : 'total_parti_pledge'}, axis =1 , inplace = True)
plepar = plepar.drop(['day'], axis = 1)

testdata2 = pd.merge(testdata2 , plepar , on= 'acc_id' , how = 'left')
#######################

mean_playtime = testdata2.groupby(['acc_id']).mean().reset_index()[['acc_id','playtime']]
mean_playtime.rename({'playtime' : 'mean_playtime'} , axis =1, inplace = True )
testdata2 = pd.merge(testdata2, mean_playtime, on ='acc_id', how = 'left')



##########
pledge_num = testdata2[['acc_id','pledge_id']].drop_duplicates().dropna().groupby('acc_id').count().reset_index()
pledge_num.rename({'pledge_id' : 'pledge_num'} , axis =1, inplace = True )
testdata2 =pd.merge(testdata2, pledge_num, on= 'acc_id' , how = 'left')


###########

pledge_size = testdata2[['acc_id','pledge_id']].drop_duplicates().dropna().groupby('pledge_id').count().reset_index()
pledge_size.rename({'acc_id' : 'member_num'}, axis= 1 , inplace = True)

mem = testdata2[['acc_id','pledge_id']].drop_duplicates().dropna()
mem = pd.merge(mem, pledge_size, on ='pledge_id', how = 'left')
mem = mem.groupby('acc_id').sum().reset_index()

testdata2 = pd.merge(testdata2, mem , on='acc_id', how = 'left')

testdata2['pledge_num'].fillna(0, inplace= True)
testdata2['member_num'].fillna(0, inplace= True)


con = testdata2[['day','acc_id']].drop_duplicates() # 얼마나 몰아서 했는가

con['Data_lagged'] = con.groupby(['acc_id'])['day'].shift(1)
con['diff'] = con['day'] - con['Data_lagged']
con.fillna(0 ,inplace= True)

con = con[con['diff']<=1]

con = con.groupby('acc_id').sum().reset_index()
con['ratioConsecutive'] = con['diff'] / 27 # 27?????????????????????????????????????????????????????????????????????
con.drop(['day','Data_lagged','diff'], axis =1 , inplace = True)

testdata2 = pd.merge(testdata2, con, on = ['acc_id'])


tot = testdata2.groupby('acc_id').sum().reset_index()
tot = tot[['acc_id','playtime']]
tot.rename({'playtime' : 'total_playtime'}, axis = 1, inplace= True)

testdata2 = pd.merge(testdata2, tot, on = ['acc_id'])

#sns.boxplot(  y= 'total_playtime' , x= "suv", data= data)


timedelta = testdata2.groupby(['day','acc_id']).sum().reset_index() # 유저별 캐릭터별 total playtime 
timedelta = timedelta[['day','acc_id','playtime']]
timedelta['day_lagged'] = timedelta.groupby(['acc_id'])['day'].shift(1)
timedelta['total_playtime_lagged'] = timedelta.groupby(['acc_id'])['playtime'].shift(1)

timedelta['day_diff'] = timedelta['day'] - timedelta['day_lagged']
timedelta['playtime_diff'] = timedelta['playtime'] - timedelta['total_playtime_lagged']
#.timedelta[timedelta['acc_id'] == '75001']

timedelta = timedelta.groupby('acc_id').sum().reset_index()
timedelta['total_playtime_changeAve'] = timedelta['playtime_diff'] / timedelta['day_diff']
timedelta = timedelta.fillna(0)[['acc_id','total_playtime_changeAve']]

testdata2 = pd.merge(testdata2, timedelta, on = ['acc_id'])
#sns.boxplot(  y= 'total_playtime_changeAve' , x= "suv", data= data)

# 변화1 + 변화2 + 변화3 / 3

'''
combatvar = testdata2[['acc_id','pledge_cnt','temp_cnt','same_pledge_cnt','etc_cnt']].groupby('acc_id').var().reset_index()
combatvar['combatvar'] = combatvar['pledge_cnt'] + combatvar['temp_cnt'] + combatvar['same_pledge_cnt'] + combatvar['etc_cnt']
combatvar = combatvar[['acc_id','combatvar']]

testdata2 = pd.merge(testdata2, combatvar)'''


source_type = test2_trade.groupby('source_acc_id').sum().reset_index()[['source_acc_id','type']]
target_type = test2_trade.groupby('target_acc_id').sum().reset_index()[['target_acc_id','type']]

target_type.rename({'target_acc_id' : 'acc_id', 'type' : 'type1'} , axis= 1 , inplace = True)
source_type.rename({'source_acc_id' : 'acc_id'} , axis= 1 , inplace = True)

exchange = pd.merge(target_type, source_type, on = 'acc_id', how = 'outer')
exchange['type'].fillna(0, inplace = True)
exchange['type1'].fillna(0, inplace = True)

exchange['num_exchange'] = exchange['type1'] + exchange['type']

exchange = exchange[['acc_id','num_exchange']]
testdata2 = pd.merge(testdata2, exchange, on = 'acc_id', how = 'left')
testdata2['num_exchange'].fillna(0, inplace = True)


total_source_type = test2_trade.groupby('source_acc_id').count().reset_index()[['source_acc_id','type']]
total_target_type = test2_trade.groupby('target_acc_id').count().reset_index()[['target_acc_id','type']]

total_target_type.rename({'target_acc_id' : 'acc_id', 'type' : 'type1'} , axis= 1 , inplace = True)
total_source_type.rename({'source_acc_id' : 'acc_id'} , axis= 1 , inplace = True)

total_trade = pd.merge(total_source_type, total_target_type, on ='acc_id', how = 'outer')
total_trade['type'].fillna(0, inplace = True)
total_trade['type1'].fillna(0, inplace = True)

total_trade['total_trade'] = total_trade['type'] + total_trade['type1']
total_trade = total_trade[['acc_id','total_trade']]

testdata2 = pd.merge(testdata2 , total_trade, on='acc_id' , how = 'left')
testdata2['total_trade'].fillna(0, inplace = True)

testdata2['num_private_shop'] = testdata2['total_trade'] - testdata2['num_exchange']
testdata2['ratio_num_exchange'] = testdata2['num_exchange'] / testdata2['total_trade']
testdata2['ratio_num_private_shop'] = testdata2['num_private_shop'] / testdata2['total_trade']

testdata2['ratio_num_exchange'].fillna(0, inplace = True)
testdata2['ratio_num_private_shop'].fillna(0, inplace = True)


# 각 유저마다 특정 혈맹에 연속적으로 참여하는 비율을 구하고 그 비율을 특정혈맹으로 묶어 평균 (특정 혈맹에 연속적으로 참여하는 평균 비율)
# 내가 속한 혈맹의 참여율
pledgeConsecutive = testdata2[['day','acc_id','pledge_id']].drop_duplicates().dropna() 
pledgeConsecutive['day_lagged'] = pledgeConsecutive.groupby(['acc_id','pledge_id'])['day'].shift(1)
#pledgeConsecutive[(pledgeConsecutive['acc_id'] == '75001') & (pledgeConsecutive['pledge_id'] == '12443.0')]

pledgeConsecutive['diff'] = pledgeConsecutive['day'] - pledgeConsecutive['day_lagged']
pledgeConsecutive['diff'].fillna(0, inplace = True)
pledgeConsecutive = pledgeConsecutive[pledgeConsecutive['diff']<=1]
pledgeConsecutive = pledgeConsecutive.groupby(['acc_id','pledge_id']).sum().reset_index()

pledgeConsecutive['pledge_ratioConsecutive'] = pledgeConsecutive['diff'] / 27
pledgeConsecutive = pledgeConsecutive[['acc_id','pledge_id','pledge_ratioConsecutive']]

pledgeConsecutive = pledgeConsecutive.groupby('pledge_id').mean().reset_index()

testdata2 = pd.merge(testdata2 , pledgeConsecutive, on ='pledge_id', how = 'left')

mean_pledge_ratioConsecutive = testdata2[['acc_id','pledge_id','pledge_ratioConsecutive']].drop_duplicates().dropna()
mean_pledge_ratioConsecutive = mean_pledge_ratioConsecutive.groupby('acc_id').mean().reset_index()

mean_pledge_ratioConsecutive.rename({'pledge_ratioConsecutive' : 'mean_pledge_ratioConsecutive'}, axis =1 , inplace = True)
testdata2 = pd.merge(testdata2 , mean_pledge_ratioConsecutive, on ='acc_id', how ='left')
testdata2['mean_pledge_ratioConsecutive'].fillna(0, inplace = True)


# 유저당 캐릭터 개수
char = testdata2[['acc_id','char_id']].drop_duplicates().groupby('acc_id').count().reset_index()
char.rename({'char_id': 'num_char'}, axis =1 , inplace = True)
testdata2 = pd.merge(testdata2 , char , on ='acc_id' , how ='left')


#플레이 횟수
playnum = testdata2.groupby('acc_id').count().reset_index()[['acc_id','day']]
playnum.rename({'day' : 'playnum'} , axis =1 , inplace = True)
testdata2 = pd.merge(testdata2 , playnum , on  = 'acc_id', how = 'left')

#몬스터 타격 횟수
testdata2['rich_monster'] = testdata2['rich_monster'].astype(int)

rich_monster = testdata2.groupby('acc_id').sum().reset_index()[['acc_id','rich_monster']]
rich_monster.rename({'rich_monster' : 'num_rich_monster'}, axis=1 ,inplace = True)
testdata2 = pd.merge(testdata2, rich_monster , on ='acc_id', how = 'left')


# level 변화의 총합

testdata2['level'] = testdata2['level'].astype(float)
a = testdata2[['acc_id','char_id','level']]
level_max = a.groupby(['acc_id','char_id']).max().reset_index()
level_max.rename({'level' : 'max_level'} , axis =1 , inplace = True)
level_min = a.groupby(['acc_id','char_id']).min().reset_index()
level_min.rename({'level' : 'min_level'} , axis =1 , inplace = True)

level = pd.merge(level_max, level_min, on =['acc_id','char_id']) 
level['level_change'] = level['max_level'] - level['min_level']
level = level[['acc_id','char_id','level_change']]
level = level.groupby('acc_id').sum().reset_index()

testdata2 = pd.merge(testdata2, level, on ='acc_id', how = 'left')

In [96]:
lst = data.columns.tolist()
lst2 = data.columns.tolist()
#playtime solo_exp quest_exp fishing private_shop pledge_cnt play_char_cnt revenue_item_price  trade_mean total_parti_pledge 
#mean_playtime ratioConsecutive total_playtime   num_exchange total_trade ratio_num_private_shop  days pledge_ratioConsecutive
#playnum
nouse = ['day',
  'class','level','rich_monster',
 'char_id',
 'server',
 'pledge_ratioConsecutive',


 
 'pledge_id',
  'suv']

for i in nouse :
        lst.remove(i)
        
nouse2 = ['day',
 'class','level','rich_monster',
 'char_id',
 'server',
 'pledge_ratioConsecutive',
 
 'pledge_id',
  'suv']

for i in nouse2 :
        lst2.remove(i)
        
data['acc_id'] = data['acc_id'].astype(int)
testdata1['acc_id'] = testdata1['acc_id'].astype(int)
testdata2['acc_id'] = testdata2['acc_id'].astype(int)

new_data = pd.get_dummies(data[lst])
new_testdata1 = pd.get_dummies(testdata1[lst])
new_testdata2 = pd.get_dummies(testdata2[lst])

KeyError: "['survival_time' 'amount_spent' 'level_change_x' 'level_change_y'] not in index"

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

suv_train_x = new_data.groupby('acc_id').mean().reset_index()[lst].values                 #mean학습
suv_train_y = new_data.groupby(['acc_id']).mean().reset_index()['survival_time'].values

spen_train_x = new_data.groupby('acc_id').mean().reset_index()[lst2].values
spen_train_y = new_data.groupby(['acc_id']).mean().reset_index()['amount_spent'].values


test_su_x1= new_testdata1.groupby(['acc_id']).mean().reset_index()[lst].values
test_su_x2= new_testdata2.groupby(['acc_id']).mean().reset_index()[lst].values

test_sp_x1= new_testdata1.groupby(['acc_id']).mean().reset_index()[lst2].values
test_sp_x2= new_testdata2.groupby(['acc_id']).mean().reset_index()[lst2].values



suv_lr = xgb.XGBRegressor(booster='gbtree', eta = 0.1, min_child_weight =5,
                          max_depth = 5, gamma = 0 , alpha = 5, n_estimators=200).fit(suv_train_x,suv_train_y)

spen_lr = xgb.XGBRegressor(booster='gbtree', eta = 0.1, min_child_weight =5,
                          max_depth = 5, gamma = 0 , alpha = 5, n_estimators=200).fit(spen_train_x,spen_train_y)


su = suv_lr.predict(test_su_x1)
sp = spen_lr.predict(test_sp_x1)

su2 = suv_lr.predict(test_su_x2)
sp2 = spen_lr.predict(test_sp_x2)

ccc = testdata1.groupby('acc_id').mean().reset_index()[['acc_id']]

aaa = pd.DataFrame(su)
aaa.rename({0: 'survival_time'}, axis =1 , inplace = True)

bbb = pd.DataFrame(sp)
bbb.rename({0: 'amount_spent'}, axis =1 , inplace = True)

file1 = pd.concat([ccc,aaa,bbb], axis =1 )


ddd = testdata2.groupby('acc_id').mean().reset_index()[['acc_id']]

aaa2 = pd.DataFrame(su2)
aaa2.rename({0: 'survival_time'}, axis =1 , inplace = True)

bbb2 = pd.DataFrame(sp2)
bbb2.rename({0: 'amount_spent'}, axis =1 , inplace = True)

file2 = pd.concat([ddd,aaa2,bbb2], axis =1 )

file1['amount_spent'] = file1['amount_spent'].apply(lambda x : 0 if x<0 else x)
file2['amount_spent'] = file2['amount_spent'].apply(lambda x : 0 if x<0 else x)

file1['survival_time'] = file1['survival_time'].apply(lambda x : 1 if x<1 else x)
file2['survival_time'] = file2['survival_time'].apply(lambda x : 1 if x<1 else x)


file1.to_csv('C:/Users/SS/Desktop/test1_predict.csv' ,header= True, index= False, encoding='UTF-8')

file2.to_csv('C:/Users/SS/Desktop/test2_predict.csv' ,header= True, index= False, encoding='UTF-8')

# modeling

In [ ]:
lst = data.columns.tolist()
lst2 = data.columns.tolist()
#playtime solo_exp quest_exp fishing private_shop pledge_cnt play_char_cnt revenue_item_price  trade_mean total_parti_pledge 
#mean_playtime ratioConsecutive total_playtime   num_exchange total_trade ratio_num_private_shop  days pledge_ratioConsecutive
#playnum
nouse = ['day',
 'acc_id',
 'char_id',
 'server',
 
 'npc_kill',
 
 'party_exp',
 
 'rich_monster',
 'death',
 'revive',
 'exp_recovery',
 
 
 'game_money_change',
 'enchant_count',
 'survival_time',
 'amount_spent',
 'class',
 'level',
 
 'random_attacker_cnt',
 'random_defender_cnt',
 'temp_cnt',
 'same_pledge_cnt',
 'etc_cnt',
 'num_opponent',
 'pledge_id',
 
 'combat_char_cnt',
 'pledge_combat_cnt',
 'random_attacker_cnt_p',
 'random_defender_cnt_p',
 'same_pledge_cnt_p',
 'temp_cnt_p',
 'etc_cnt_p',
 'combat_play_time',
 'non_combat_play_time',
 'day_spent',
 'suv',
 'trade',
 'give_item_amount',
 
 'receive_item_amount',
 'spend_item_price',
 'total_item_amount',
 'var_kindpartner_day',

 

 'pledge_num',
 'member_num',
 
 
 'total_playtime_changeAve',
 
 
 'num_private_shop',
 'ratio_num_exchange',
 
 
 
 'mean_pledge_ratioConsecutive',
 'num_char']

for i in nouse :
        lst.remove(i)
        
nouse2 = ['private_shop',
 'random_defender_cnt',
 'party_exp',
 'combat_char_cnt',
 'pledge_num',
 'ratio_num_private_shop',
 'temp_cnt',
 'enchant_count',
 'revive',
 'exp_recovery',
 'mean_playtime', 'day', 'acc_id','char_id', 'server','rich_monster','pledge_id' ,'suv' ,'trade','pledge_ratioConsecutive' ,'class', 'level','survival_time','amount_spent']

for i in nouse2 :
        lst2.remove(i)
        
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

suv_train_x = data.groupby(['acc_id']).mean().reset_index()[lst].values                 #mean학습
suv_train_y = data.groupby(['acc_id']).mean().reset_index()['survival_time'].values

spen_train_x = data.groupby(['acc_id']).mean().reset_index()[lst2].values
spen_train_y = data.groupby(['acc_id']).mean().reset_index()['amount_spent'].values
 
#suv_train_x = data[lst].values                      #그냥 다 학습
#suv_train_y = data['survival_time'].values

#spen_train_x = data.groupby('acc_id').mean().reset_index()[lst].values
#spen_train_y = data.groupby('acc_id').mean().reset_index()['amount_spent'].values


test_su_x1= testdata1.groupby(['acc_id']).mean().reset_index()[lst].values
test_su_x2= testdata2.groupby(['acc_id']).mean().reset_index()[lst].values

test_sp_x1= testdata1.groupby(['acc_id']).mean().reset_index()[lst2].values
test_sp_x2= testdata2.groupby(['acc_id']).mean().reset_index()[lst2].values

#suv_lr = RandomForestRegressor(n_estimators = 100).fit(suv_train_x, suv_train_y)
#spen_lr = RandomForestRegressor(n_estimators = 100).fit(spen_train_x, spen_train_y)

suv_lr = xgb.XGBRegressor(booster='gbtree', eta = 0.1, min_child_weight =5,
                          max_depth = 5, gamma = 0 , alpha = 5, n_estimators=200).fit(suv_train_x,suv_train_y)

spen_lr = xgb.XGBRegressor(booster='gbtree', eta = 0.1, min_child_weight =5,
                          max_depth = 5, gamma = 0 , alpha = 5, n_estimators=200).fit(spen_train_x,spen_train_y)


su = suv_lr.predict(test_su_x1)
sp = spen_lr.predict(test_sp_x1)

su2 = suv_lr.predict(test_su_x2)
sp2 = spen_lr.predict(test_sp_x2)

ccc = testdata1.groupby('acc_id').mean().reset_index()[['acc_id']]

aaa = pd.DataFrame(su)
aaa.rename({0: 'survival_time'}, axis =1 , inplace = True)

bbb = pd.DataFrame(sp)
bbb.rename({0: 'amount_spent'}, axis =1 , inplace = True)

file1 = pd.concat([ccc,aaa,bbb], axis =1 )


ddd = testdata2.groupby('acc_id').mean().reset_index()[['acc_id']]

aaa2 = pd.DataFrame(su2)
aaa2.rename({0: 'survival_time'}, axis =1 , inplace = True)

bbb2 = pd.DataFrame(sp2)
bbb2.rename({0: 'amount_spent'}, axis =1 , inplace = True)

file2 = pd.concat([ddd,aaa2,bbb2], axis =1 )

file1['amount_spent'] = file1['amount_spent'].apply(lambda x : 0 if x<0 else x)
file2['amount_spent'] = file2['amount_spent'].apply(lambda x : 0 if x<0 else x)

file1['survival_time'] = file1['survival_time'].apply(lambda x : 1 if x<1 else x)
file2['survival_time'] = file2['survival_time'].apply(lambda x : 1 if x<1 else x)


file1.to_csv('C:/Users/SS/Desktop/test1_predict.csv' ,header= True, index= False, encoding='UTF-8')

file2.to_csv('C:/Users/SS/Desktop/test2_predict.csv' ,header= True, index= False, encoding='UTF-8')


KeyboardInterrupt

